In [11]:
import pandas as pd
import spacy

# Cargar modelo de spaCy (español)
nlp = spacy.load('es_core_news_sm')

# Leer el CSV
df_comparado = pd.read_csv("comparacion_csv_psalmos_def.csv", sep=";", encoding="UTF-8")


from difflib import SequenceMatcher

def diferencia_caracteres(texto1, texto2):
    if not isinstance(texto1, str) or not isinstance(texto2, str):
        return ""
    
    matcher = SequenceMatcher(None, texto1, texto2)
    resultado = []

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == 'equal':
            resultado.append(texto1[i1:i2])
        elif tag == 'replace':
            resultado.append(f"[{texto1[i1:i2]} ➝ {texto2[j1:j2]}]")
        elif tag == 'delete':
            resultado.append(f"[-{texto1[i1:i2]}-]")
        elif tag == 'insert':
            resultado.append(f"[+{texto2[j1:j2]}+]")

    return ''.join(resultado)




/Users/castan0000/anaconda3/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'es_core_news_sm' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.8.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [12]:
from difflib import SequenceMatcher
import pandas as pd

def diferencias_palabras_completas(texto1, texto2, dtype="object"):
    if not isinstance(texto1, str) or not isinstance(texto2, str):
        return "", ""

    matcher = SequenceMatcher(None, texto1.split(), texto2.split())
    diferencias_v1 = []
    diferencias_v2 = []

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag in ['replace', 'delete']:
            for w1 in texto1.split()[i1:i2]:
                if w1.strip():
                    diferencias_v1.append(w1)
        if tag in ['replace', 'insert']:
            for w2 in texto2.split()[j1:j2]:
                if w2.strip():
                    diferencias_v2.append(w2)

    # Analizar cambios gráficos (alta similitud)
    cambios_v1 = []
    cambios_v2 = []

    for w1 in diferencias_v1:
        mejores_coincidencias = [
            (w2, SequenceMatcher(None, w1, w2).ratio()) for w2 in diferencias_v2
        ]
        mejores_coincidencias.sort(key=lambda x: -x[1])  # más parecido primero
        if mejores_coincidencias and mejores_coincidencias[0][1] >= 0.7:
            w2_similar, ratio = mejores_coincidencias[0]
            diferencias_v2.remove(w2_similar)
            cambios_v1.append(f"{w1} ⚠️")
            cambios_v2.append(f"{w2_similar} ⚠️")
        else:
            cambios_v1.append(w1)

    # Lo que quede en diferencias_v2 no emparejado
    cambios_v2.extend(diferencias_v2)

    return " | ".join(cambios_v1), " | ".join(cambios_v2)

df_comparado["Text_V1"] = df_comparado["Text_V1"].fillna("")
df_comparado["Text_V2"] = df_comparado["Text_V2"].fillna("")

df_comparado[['Cambios_v1', 'Cambios_v2']] = df_comparado.apply(
    lambda row: pd.Series(diferencias_palabras_completas(row["Text_V1"], row["Text_V2"])),
    axis=1
)




In [13]:
df_comparado

,Psalm_book,Chapter,Verse,Text_V1,Text_V2,Cambios_v1,Cambios_v2
0,1.0,1.0,1.0,Bien aventurado el varón que no anduvo en cons...,Byenaventurado el varon que no anduvo en conse...,Bien | aventurado ⚠️ | varón ⚠️ | los | estuvo...,Byenaventurado ⚠️ | varon ⚠️ | estuvo: ⚠️ | as...
1,1.0,1.0,2.0,"Mas sólo en ley de Adonay su voluntad, y en s...",Mas solo en ley de .A. su voluntad: y en su le...,"sólo ⚠️ | Adonay | voluntad, ⚠️ | ley | fablar...",solo ⚠️ | voluntad: ⚠️ | .A. | lei | pensara |...
2,1.0,1.0,3.0,Y será como árbol plantado sobre piélagos de a...,Y sera como arbol plantado cerca pielagos de a...,será ⚠️ | árbol ⚠️ | sobre | piélagos ⚠️ | agu...,sera ⚠️ | arbol ⚠️ | pielagos ⚠️ | aguas: ⚠️ |...
3,1.0,1.0,4.0,"No assí los malos, sino como tamo que lo empu...",No assi los malos: sino como tamo que lo empux...,"assí ⚠️ | malos, ⚠️",assi ⚠️ | malos: ⚠️
4,1.0,1.0,5.0,"Por tanto, no se alevantarán malos en juizio,...",Por tanto no se alevantaran malos en juyzio: n...,"tanto, ⚠️ | alevantarán ⚠️ | juizio, ⚠️",tanto ⚠️ | alevantaran ⚠️ | juyzio: ⚠️
...,...,...,...,...,...,...,...
2539,5.0,150.0,3.0,"Alabaldo con sonido de sophar, alabaldo con g...",Alabaldo con sonido de sofar alabaldo con gayt...,"sophar,",sofar
2540,5.0,150.0,4.0,"Alabaldo con adufle y bayle, alabaldo con est...",Alabaldo con adufie y bayle: alabaldo con estr...,"adufle ⚠️ | bayle, ⚠️ | órgano. ⚠️",adufie ⚠️ | bayle: ⚠️ | organo. ⚠️
2541,5.0,150.0,5.0,"Alabaldo con reteñideras de oída, alabaldo co...",Alabaldo con reteñideras de oyda: alabaldo con...,"oída, | aublación. ⚠️",aublacion. ⚠️ | oyda:
2542,5.0,150.0,6.0,Toda alma alabe a Adonay. Halelu-Yah.,Toda alma alabe a .A.: Halelu Yah.,Adonay. | Halelu-Yah. ⚠️,Halelu ⚠️ | .A.: | Yah.


In [14]:
df_comparado

,Psalm_book,Chapter,Verse,Text_V1,Text_V2,Cambios_v1,Cambios_v2
0,1.0,1.0,1.0,Bien aventurado el varón que no anduvo en cons...,Byenaventurado el varon que no anduvo en conse...,Bien | aventurado ⚠️ | varón ⚠️ | los | estuvo...,Byenaventurado ⚠️ | varon ⚠️ | estuvo: ⚠️ | as...
1,1.0,1.0,2.0,"Mas sólo en ley de Adonay su voluntad, y en s...",Mas solo en ley de .A. su voluntad: y en su le...,"sólo ⚠️ | Adonay | voluntad, ⚠️ | ley | fablar...",solo ⚠️ | voluntad: ⚠️ | .A. | lei | pensara |...
2,1.0,1.0,3.0,Y será como árbol plantado sobre piélagos de a...,Y sera como arbol plantado cerca pielagos de a...,será ⚠️ | árbol ⚠️ | sobre | piélagos ⚠️ | agu...,sera ⚠️ | arbol ⚠️ | pielagos ⚠️ | aguas: ⚠️ |...
3,1.0,1.0,4.0,"No assí los malos, sino como tamo que lo empu...",No assi los malos: sino como tamo que lo empux...,"assí ⚠️ | malos, ⚠️",assi ⚠️ | malos: ⚠️
4,1.0,1.0,5.0,"Por tanto, no se alevantarán malos en juizio,...",Por tanto no se alevantaran malos en juyzio: n...,"tanto, ⚠️ | alevantarán ⚠️ | juizio, ⚠️",tanto ⚠️ | alevantaran ⚠️ | juyzio: ⚠️
...,...,...,...,...,...,...,...
2539,5.0,150.0,3.0,"Alabaldo con sonido de sophar, alabaldo con g...",Alabaldo con sonido de sofar alabaldo con gayt...,"sophar,",sofar
2540,5.0,150.0,4.0,"Alabaldo con adufle y bayle, alabaldo con est...",Alabaldo con adufie y bayle: alabaldo con estr...,"adufle ⚠️ | bayle, ⚠️ | órgano. ⚠️",adufie ⚠️ | bayle: ⚠️ | organo. ⚠️
2541,5.0,150.0,5.0,"Alabaldo con reteñideras de oída, alabaldo co...",Alabaldo con reteñideras de oyda: alabaldo con...,"oída, | aublación. ⚠️",aublacion. ⚠️ | oyda:
2542,5.0,150.0,6.0,Toda alma alabe a Adonay. Halelu-Yah.,Toda alma alabe a .A.: Halelu Yah.,Adonay. | Halelu-Yah. ⚠️,Halelu ⚠️ | .A.: | Yah.


In [17]:
df_final = df_comparado[["Chapter", "Verse", "Text_V1", "Text_V2", "Cambios_v1", "Cambios_v2"]]
df_final.to_csv("comparacion_salmos_nueva_version_def.csv", index=False)
